In [1]:
import xgboost as xgb
import numpy as np
import pickle
import pandas as pd
import os
import csv
import matplotlib.pyplot as plt

from datetime import timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Especificar ruta del modelo y DataFrame

In [33]:
ruta_modelo = "C:\\Users\\franc\\OneDrive\\Documentos\\VScode\\Github\\Franico-proyecto\\models\\XGBoost_10000.pkl"

df = pd.read_excel("C:\\Users\\franc\\OneDrive\\Documentos\\VScode\\Github\\Franico-proyecto\\models\\df_filtered_test.xlsx")

In [35]:
class ClippingDetector:

    def __init__(self, df):

        #Carga el DataFrame a una variable interna
        self.data_F = df

        # Corre las horas
        #self.data_F.time = self.data_F.time - timedelta(hours=4) 

        self.data_F['fecha'] = self.data_F['time'].dt.date
        self.data_F['time'] = pd.DatetimeIndex(self.data_F['time'])

    def load_model(self, ruta_modelo):

        #Cargar modelo
        with open(ruta_modelo, "rb") as archivo:
            self.modelo = pickle.load(archivo)


    '''Funciones internas'''

    #Extrae información del dataframe
    def extract_data(self):
        
        fecha_plantas = self.data_F["fecha"].unique()
        
        pt_pr = []

        for f in fecha_plantas:
            dt = self.data_F[self.data_F["fecha"] == f]
            pt = list(dt["Potencia Teórica"].values)          #Potencia Teórica
            pr = list(dt["Potencia Fotovoltaica"].values)     #Potencia Real
            pt_pr.append([pt, pr])                            #Lista con los 3 vectores anteriores

        '''Retorna una lista de listas con la potencia teórica y la potencia real, respectivamente.'''
        return pt_pr
    
    
    #Normaliza las curvas de generación y poa
    def normalize_data(self, lista):
    
        new_list = [ [x[0], x[1]] for x in lista ]
        self.normalized_data = []

        for x in new_list:
            maximo = max(max(x[0]), max(x[1]))
            x1  = [0 if i < 0 else i for i in x[0]]
            x2  = [0 if i < 0 else i for i in x[1]]
        
            x1 = x1/maximo
            x2 = x2/maximo

            self.normalized_data.append([x1,x2])

        '''Retorna una lista de listas con la potencia teórica y la potencia real, ambas normalizadas, respectivamente.'''
        return self.normalized_data
    

    def preparation(self):
   
        data_list = self.extract_data()
        self.ndata_list = self.normalize_data(data_list)

        # Genera un vector con ambas generaciones concatenadas (len: 64)
        self.flatten_data = [ list(x[0][12:-4]) + list(x[1][12:-4]) for x in self.ndata_list ]


    def clipping_predict(self):

        self.dm_data = xgb.DMatrix(data = self.flatten_data)

        self.pred_prob = self.modelo.predict(self.dm_data)
        self.clipping = []

        for y in self.pred_prob:
            cpred = [1 if i >= 0.44 else 0 for i in y]
            self.clipping.append(cpred)

        #Agregar los 0 faltantes

        l12 = [0]*12
        l4 = [0]*4    

        extended_clipping = []
        for clp in self.clipping:
            extended_clipping.extend(l12 + clp + l4)

        self.data_F['clipping2'] = extended_clipping
        
        
        return self.clipping

In [36]:
modelo = ClippingDetector(df)
modelo.load_model(ruta_modelo)

modelo.preparation()
modelo.clipping_predict()

# DataFrame final con clipping
dataframe = modelo.data_F

In [37]:
dataframe

,time,Potencia Teórica,Potencia proyectada con Soiling,Potencia Fotovoltaica,Consumo de Red,Consumo Total,Unnamed: 6,fecha,clipping2
0,2023-08-01 00:00:00,2.124040,0.182,0.181667,121.334270,121.515937,NaN,2023-08-01,0
1,2023-08-01 00:30:00,18.598711,2.849,2.848500,138.379299,141.227799,NaN,2023-08-01,0
2,2023-08-01 01:00:00,30.492730,13.811,13.811334,108.694374,122.505708,NaN,2023-08-01,0
3,2023-08-01 01:30:00,54.587289,30.765,30.764666,129.121452,159.886118,NaN,2023-08-01,0
4,2023-08-01 02:00:00,75.164539,51.435,51.434667,126.383561,177.818228,NaN,2023-08-01,0
...,...,...,...,...,...,...,...,...,...
1435,2023-08-30 21:30:00,0.000000,0.000,0.000000,118.521843,118.521843,NaN,2023-08-30,0
1436,2023-08-30 22:00:00,0.000000,0.000,0.000000,128.425396,128.425396,NaN,2023-08-30,0
1437,2023-08-30 22:30:00,0.000000,0.000,0.000000,114.101002,114.101002,NaN,2023-08-30,0
1438,2023-08-30 23:00:00,0.000000,0.000,0.000000,118.483477,118.483477,NaN,2023-08-30,0
